In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Hyper-parameters 
input_size = 28 * 28    # 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [4]:
# MNIST dataset (images and labels)
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

100.0%


Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw



100.0%


Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw



100.0%


Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw



100.0%

Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw



In [5]:
# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [6]:
# Logistic regression model
model = nn.Linear(input_size, num_classes)

In [7]:
# Loss and optimizer
# nn.CrossEntropyLoss() computes softmax internally
criterion = nn.CrossEntropyLoss()  # takes (logits, labels) as input
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [15]:
images, labels = next(iter(train_loader))
images.shape, labels.shape

(torch.Size([100, 1, 28, 28]), torch.Size([100]))

In [8]:
# Train the model
total_step = len(train_loader)  # Get the total number of batches (total steps) in the training data loader.
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, input_size)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 2.2138
Epoch [1/5], Step [200/600], Loss: 2.1029
Epoch [1/5], Step [300/600], Loss: 2.0056
Epoch [1/5], Step [400/600], Loss: 1.8978
Epoch [1/5], Step [500/600], Loss: 1.8473
Epoch [1/5], Step [600/600], Loss: 1.8023
Epoch [2/5], Step [100/600], Loss: 1.7409
Epoch [2/5], Step [200/600], Loss: 1.6713
Epoch [2/5], Step [300/600], Loss: 1.6052
Epoch [2/5], Step [400/600], Loss: 1.5283
Epoch [2/5], Step [500/600], Loss: 1.5155
Epoch [2/5], Step [600/600], Loss: 1.4665
Epoch [3/5], Step [100/600], Loss: 1.3134
Epoch [3/5], Step [200/600], Loss: 1.4265
Epoch [3/5], Step [300/600], Loss: 1.3060
Epoch [3/5], Step [400/600], Loss: 1.3032
Epoch [3/5], Step [500/600], Loss: 1.2537
Epoch [3/5], Step [600/600], Loss: 1.2952
Epoch [4/5], Step [100/600], Loss: 1.2023
Epoch [4/5], Step [200/600], Loss: 1.2723
Epoch [4/5], Step [300/600], Loss: 1.2191
Epoch [4/5], Step [400/600], Loss: 1.1111
Epoch [4/5], Step [500/600], Loss: 0.9895
Epoch [4/5], Step [600/600], Loss:

In [16]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, input_size)
        outputs = model(images) # shape: (100, 10)
        _, predicted = torch.max(outputs.data, 1)   # ouput scores, predicted labels
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the model on the 10000 test images: 82.9000015258789 %


In [ ]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')